In [7]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import pandas as pd

# Replace 'path/to/chromedriver' with the path to your chromedriver executable
path = "C:/Users/jyoti/OneDrive/Documents/01_Data_Analytics/CommunityDreams/chromedriver/chromedriver.exe"
#driver = webdriver.Chrome(executable_path=path)

# Create ChromeOptions object
chrome_options = webdriver.ChromeOptions()

# Set preference to not load images
chrome_options.add_experimental_option("prefs", {"profile.managed_default_content_settings.images": 2})

# Pass ChromeOptions to the webdriver
driver = webdriver.Chrome(executable_path=path, options=chrome_options)

# Open the URL
driver.get("https://www.pbcgov.org/papa/Asps/GeneralAdvSrch/SearchPage.aspx?f=a")
driver.maximize_window()
# Wait for the page to load
wait = WebDriverWait(driver, 10)

# Wait for the page to load
parcelIds = []
legalDescriptions = []
owners = []
addresses1 = []
addresses2 = []
marketPrices = []
County = 'Palm Beach'

# Find the ZIP code input field and enter '334'
zip_code_input = driver.find_element(By.NAME, "ctl00$MainContent$txtZip")
zip_code_input.clear()
zip_code_input.send_keys("334")

search_button = driver.find_element(By.NAME,"ctl00$MainContent$btnAdvSrch")
search_button.click()
#driver.implicitly_wait(15)

# Change page size to 100 records
page_size_dropdown = driver.find_element(By.NAME, "ctl00$MainContent$ddlPgSizeTop")
page_size_dropdown.send_keys("100")

parcelIds = []
legalDescriptions = []
owners = []
addresses1 = []
addresses2 = []
marketPrices = []
County = 'Palm Beach'

#startfromPage = 10
#counter = 0
no_of_pages = 300

page_link = driver.find_element(By.XPATH, f"//a[text()='{startfromPage}']")
#page_link.click()

# Iterate through pages 2 to 10
#while counter < no_of_pages:
    # Click on the page link
    

results_table = driver.find_element(By.ID, "gvSrchResults")
rows = results_table.find_elements(By.XPATH, "//tr[contains(@class, 'gridrow')]")

result_urls = []
for row in rows:
    clickable_element = row.find_element(By.XPATH, ".//td[@onclick]")
    result_urls.append(clickable_element.get_attribute("onclick"))

# Visit each result page separately
for result_url in result_urls:
    driver.execute_script(result_url)
    try:
        parcelid = driver.find_element(By.ID, "MainContent_lblPCN").get_attribute("textContent")
        parcelIds.append(parcelid)
    except Exception as e:
        parcelIds.append(None)


    try:
        legalDescription = driver.find_element(By.ID, "MainContent_lblLegalDesc").get_attribute("textContent")
        legalDescriptions.append(legalDescription)
    except Exception as e:
        legalDescriptions.append(None)


    try:
        owner_elements = driver.find_elements(By.XPATH, '//*[@id="ownerInformationDiv"]/fieldset/table/tbody/tr[2]/td[1]/table/tbody/tr[position() > 1]/td')
        owners_list = [owner.text for owner in owner_elements]
        owner = ' & '.join(owners_list)
        owners.append(owner)
    except Exception as e:
        owners.append(None)


    try:
        address1 = driver.find_element(By.ID, "MainContent_lblAddrLine1").get_attribute("textContent")
        address2 = driver.find_element(By.ID, "MainContent_lblAddrLine3").get_attribute("textContent")

        addresses1.append(address1)
        addresses2.append(address2)
    except Exception as e:
        addresses1.append(None)
        addresses2.append(None)

    try:
        marketPrice = driver.find_element(By.ID, "MainContent_lblMarketValue1").get_attribute("textContent")
        marketPrices.append(marketPrice)
    except Exception as e:
        marketPrices.append(None)

    driver.implicitly_wait(1)
    driver.back()

pageCompleted = startfromPage+counter
print(f'PageNo {pageCompleted} completed')
counter += 1
 # Click the "Next" button
next_button_locator = (By.XPATH, "//a[text()='Next']")
try:
    next_button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable(next_button_locator))
    next_button.click()
except:
    break  # Break if there is no next button

# Close the driver
driver.quit()




PageNo 102 completed


SyntaxError: 'break' outside loop (1930336464.py, line 130)

In [8]:
df = pd.DataFrame({
    'ParcelId': parcelIds,
    'Owner Name': owners,
    'Property Address': addresses1,
    'Address':addresses2,
    'County':County,
    'Property Value': marketPrices,
    'Land Use': legalDescriptions
})

df.to_csv('Palm_beach_final_raw_data/rawData_pg_1_FL_palm_beach_county.csv', index=False)